In [153]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import time
import math
import operator

## 数据处理

In [8]:
orders = pd.read_csv('./tm_online_data/orders.csv',encoding='gbk')
items_orders = pd.read_csv('./tm_online_data/Items_orders.csv',encoding='gbk')
items_attribute = pd.read_csv('./tm_online_data/Items_attribute.csv',encoding='gbk')

In [9]:
orders.head()

,订单编号,买家会员名,买家支付宝账号,买家应付货款,买家应付邮费,买家支付积分,总金额,返点积分,买家实际支付金额,买家实际支付积分,...,是否代付,定金排名,修改后的sku,修改后的收货地址,异常信息,天猫卡券抵扣,集分宝抵扣,是否是O2O交易,退款金额,预约门店
0,2.140730e+16,1425,yorzikyA6C,58.51,0.0,0,58.51,0,58.51,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,2.427050e+16,2163,AC870BA5860,15.70,5.0,0,20.70,0,20.70,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,2.140260e+16,375,AC7574B65A0,7.90,5.0,0,12.90,0,12.90,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,2.139880e+16,2618,A807C90766A,4.81,5.0,0,9.81,0,9.81,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,2.144680e+16,2012,A505588565B,23.92,5.0,0,28.92,0,28.92,0,...,否,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [18]:
items_orders.head()

,订单编号,标题,价格,购买数量,外部系统编号,商品属性,套餐信息,备注,订单状态,商家编码
0,2.140730e+16,发光玩具批发光纤手指灯闪光夜市热卖货源儿童玩具地摊义乌厂家,0.58,12,WY013-2SZD0426,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426
1,2.140730e+16,特价5号AA普通干电池 电动玩具配件 厂家直销批,1.00,20,HT-5H0094,NaN,NaN,NaN,交易成功,HT-5H0094
2,2.140730e+16,大号泡泡棒沙滩小铲子工具泡泡枪公园吹泡泡户外亲子游戏玩具热卖,1.20,16,GC018005,NaN,NaN,NaN,交易成功,GC018005
3,2.140730e+16,特价正品7号电池儿童电动玩具电源配件 厂家直销1元4节地摊货批发,1.00,10,HT-7H0093,NaN,NaN,NaN,交易成功,HT-7H0093
4,2.140730e+16,儿童沙滩玩具水枪 宝宝玩水玩具户外洗澡游泳漂流戏大号水枪批发,6.79,1,GC080003,套餐类型：标准配置;颜色分类：33000蓝色水枪,NaN,NaN,交易成功,GC080003


In [20]:
items_orders[items_orders['订单编号'].isin([2.140730e+16])]['标题']

,订单编号,标题,价格,购买数量,外部系统编号,商品属性,套餐信息,备注,订单状态,商家编码
0,2.140730e+16,发光玩具批发光纤手指灯闪光夜市热卖货源儿童玩具地摊义乌厂家,0.58,12,WY013-2SZD0426,颜色分类：小号,NaN,NaN,交易成功,WY013-2SZD0426
1,2.140730e+16,特价5号AA普通干电池 电动玩具配件 厂家直销批,1.00,20,HT-5H0094,NaN,NaN,NaN,交易成功,HT-5H0094
2,2.140730e+16,大号泡泡棒沙滩小铲子工具泡泡枪公园吹泡泡户外亲子游戏玩具热卖,1.20,16,GC018005,NaN,NaN,NaN,交易成功,GC018005
3,2.140730e+16,特价正品7号电池儿童电动玩具电源配件 厂家直销1元4节地摊货批发,1.00,10,HT-7H0093,NaN,NaN,NaN,交易成功,HT-7H0093
4,2.140730e+16,儿童沙滩玩具水枪 宝宝玩水玩具户外洗澡游泳漂流戏大号水枪批发,6.79,1,GC080003,套餐类型：标准配置;颜色分类：33000蓝色水枪,NaN,NaN,交易成功,GC080003


In [132]:
df = pd.DataFrame()
for user in pd.unique(orders['买家会员名']):
    order_id = list(orders[orders['买家会员名'].isin([user])]['订单编号']) # 获取每个用户的订单，可能有多个订单
    # 根据每个订单编号找出所有物品  
    items = items_orders[items_orders['订单编号'].isin(order_id)]['标题']
    nums = items_orders[items_orders['订单编号'].isin(order_id)]['购买数量']
    df_user = pd.concat([pd.Series(items),pd.Series(nums)],axis=1)
    df_user.groupby(by='标题').apply(sum)
    df_user['user']=user
    df_user.reset_index(drop=True,inplace=True)
    df = pd.concat([df,df_user])
    df.reset_index(drop=True,inplace=True)

In [71]:
df.head()

,标题,购买数量,user
0,发光玩具批发光纤手指灯闪光夜市热卖货源儿童玩具地摊义乌厂家,12,1425
1,特价5号AA普通干电池 电动玩具配件 厂家直销批,20,1425
2,大号泡泡棒沙滩小铲子工具泡泡枪公园吹泡泡户外亲子游戏玩具热卖,16,1425
3,特价正品7号电池儿童电动玩具电源配件 厂家直销1元4节地摊货批发,10,1425
4,儿童沙滩玩具水枪 宝宝玩水玩具户外洗澡游泳漂流戏大号水枪批发,1,1425


In [126]:
itemid = []
for item in df['标题']:
    itemid.append(list(items_attribute[items_attribute['标题'].isin([item])]['宝贝ID'])) # 将标题替换成宝贝ID

In [119]:
itemid

[[530450000000.0],
 [36266985550.0],
 [544067000000.0],
 [36267739775.0],
 [548547000000.0],
 [537397000000.0],
 [537397000000.0],
 [530813000000.0],
 [545517000000.0],
 [538494000000.0],
 [520311000000.0],
 [36074765406.0],
 [521926000000.0],
 [545517000000.0],
 [543695000000.0],
 [537166000000.0],
 [36081332365.0],
 [545602000000.0],
 [539411000000.0],
 [547306000000.0],
 [547794000000.0],
 [545538000000.0],
 [536010000000.0],
 [538535000000.0],
 [544769000000.0],
 [531379000000.0],
 [550660000000.0],
 [36266985550.0],
 [544769000000.0],
 [35722000205.0],
 [35722000205.0],
 [35722000205.0],
 [550905000000.0],
 [44928090920.0],
 [44928090920.0],
 [550977000000.0],
 [545517000000.0],
 [531146000000.0],
 [522078000000.0],
 [548547000000.0],
 [547390000000.0],
 [549641000000.0],
 [549766000000.0],
 [547813000000.0],
 [542939000000.0],
 [547381000000.0],
 [547511000000.0],
 [35798985871.0],
 [35798985871.0],
 [35798985871.0],
 [35798985871.0],
 [35722000205.0],
 [537397000000.0],
 [537397

In [173]:
itemids = [] # 整理宝贝ID 空列表以Nan替代
for item_id in itemid:
    if item_id:
        itemids.append(item_id[0])
    else:
        itemids.append(np.nan)

In [175]:
df['宝贝ID']=itemids

In [176]:
df

,标题,购买数量,user,宝贝ID
0,发光玩具批发光纤手指灯闪光夜市热卖货源儿童玩具地摊义乌厂家,12,1425,5.304500e+11
1,特价5号AA普通干电池 电动玩具配件 厂家直销批,20,1425,3.626699e+10
2,大号泡泡棒沙滩小铲子工具泡泡枪公园吹泡泡户外亲子游戏玩具热卖,16,1425,5.440670e+11
3,特价正品7号电池儿童电动玩具电源配件 厂家直销1元4节地摊货批发,10,1425,3.626774e+10
4,儿童沙滩玩具水枪 宝宝玩水玩具户外洗澡游泳漂流戏大号水枪批发,1,1425,5.485470e+11
...,...,...,...,...
22505,新款回力小车军事导弹车模型儿童玩具创意小孩礼物玩具货源批发,3,2414,5.429790e+11
22506,创意宝宝玩具婴儿摇铃中号小喇叭安全材质0-1岁早教儿童玩具批,5,2414,NaN
22507,儿童沙滩玩具车套装创意礼物益智挖沙玩具 地摊货源儿童玩具批发,5,2414,5.468270e+11
22508,创意发光球闪光透明发光水晶弹力球儿童小玩具 夜市地摊货源批发,5,2414,3.572200e+10


In [181]:
# 删除标题列 删除空值
#df.drop(columns='标题',inplace=True)
df.dropna(inplace=True)

In [183]:
df.reset_index(drop=True,inplace=True)

In [184]:
df

,购买数量,user,宝贝ID
0,12,1425,5.304500e+11
1,20,1425,3.626699e+10
2,16,1425,5.440670e+11
3,10,1425,3.626774e+10
4,1,1425,5.485470e+11
...,...,...,...
20484,5,2414,5.403490e+11
20485,10,2414,5.416590e+11
20486,3,2414,5.429790e+11
20487,5,2414,5.468270e+11


In [190]:
data = [[df1['user'][i],df1['宝贝ID'][i],df1['购买数量'][i]] for i in range(len(df1))]

In [191]:
def compress(data,col):
    """
    处理data数据第col列的数据，保证此列数字是从0开始连续出现，中间不会有一个不存在此列的数字
    :param data: 二维列表数据
    :param col:处理的列
    :return:不同的个数
    """
    
    e_rows = dict()  # key是data数据中第col列数据，value是一个存放键出现在每一个行号的列表
    for i in range(len(data)):
        e = data[i][col]
        if e not in e_rows:
            e_rows[e] = []
        e_rows[e].append(i)
        
    for rows,i in zip(e_rows.values(),range(len(e_rows))):
        for row in rows:
            data[row][col] = i
    
    return len(e_rows)

In [193]:
data1= data.copy()
num_user = compress(data1,0)
num_user

3320

In [196]:
num_item = compress(data1,1)
num_item

255

In [197]:
data1

[[0, 0, 12],
 [0, 1, 20],
 [0, 2, 16],
 [0, 3, 10],
 [0, 4, 1],
 [1, 5, 1],
 [1, 5, 1],
 [2, 6, 1],
 [3, 7, 1],
 [4, 8, 8],
 [5, 9, 1],
 [5, 10, 1],
 [5, 11, 1],
 [5, 7, 1],
 [5, 12, 1],
 [5, 13, 1],
 [5, 14, 1],
 [5, 15, 1],
 [5, 16, 1],
 [6, 17, 6],
 [6, 18, 4],
 [6, 19, 2],
 [6, 20, 4],
 [6, 21, 3],
 [6, 22, 3],
 [6, 23, 2],
 [6, 24, 3],
 [7, 1, 10],
 [7, 22, 16],
 [8, 25, 60],
 [8, 25, 48],
 [8, 25, 48],
 [9, 26, 1],
 [9, 27, 1],
 [9, 27, 1],
 [9, 28, 1],
 [9, 7, 1],
 [9, 29, 1],
 [9, 30, 5],
 [9, 4, 1],
 [10, 31, 2],
 [10, 32, 1],
 [10, 33, 1],
 [10, 34, 2],
 [10, 35, 1],
 [10, 36, 1],
 [10, 37, 1],
 [10, 38, 1],
 [10, 38, 1],
 [10, 38, 1],
 [10, 38, 1],
 [11, 25, 36],
 [11, 5, 4],
 [11, 5, 4],
 [11, 5, 5],
 [11, 5, 4],
 [11, 5, 5],
 [11, 21, 5],
 [11, 1, 5],
 [12, 39, 1],
 [12, 40, 1],
 [12, 21, 1],
 [12, 41, 1],
 [13, 42, 1],
 [14, 43, 1],
 [15, 7, 3],
 [15, 7, 4],
 [16, 43, 12],
 [17, 44, 1],
 [18, 45, 3],
 [18, 46, 5],
 [18, 47, 1],
 [18, 48, 6],
 [18, 49, 5],
 [18, 49, 5],
 [

In [204]:
df1 = pd.DataFrame(data1,columns=['userid','itemid','buy_num'])
df1

,userid,itemid,buy_num
0,0,0,12
1,0,1,20
2,0,2,16
3,0,3,10
4,0,4,1
...,...,...,...
20484,3319,140,5
20485,3319,69,10
20486,3319,164,3
20487,3319,121,5


In [205]:
# 创建矩阵
df_pivot = df1.pivot_table(index='userid',columns='itemid',values='buy_num') # 重塑数据
df_pivot

itemid,0,1,2,3,4,5,6,7,8,9,...,245,246,247,248,249,250,251,252,253,254
userid,,,,,,,,,,,,,,,,,,,,,
0,12.0,20.0,16.0,10.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 封装数据处理Data类

In [89]:
class Data:
    def __init__(self):
        print('开始读取数据...')
        # 读取三个数据
        orders = pd.read_csv('./tm_online_data/orders.csv',encoding='gbk')
        items_orders = pd.read_csv('./tm_online_data/Items_orders.csv',encoding='gbk')
        items_attribute = pd.read_csv('./tm_online_data/Items_attribute.csv',encoding='gbk')
        
        df = pd.DataFrame()
        for user in pd.unique(orders['买家会员名']):
            order_id = list(orders[orders['买家会员名'].isin([user])]['订单编号']) # 获取每个用户的订单，可能有多个订单
            # 根据每个订单编号找出所有物品 及其购买量 
            items = items_orders[items_orders['订单编号'].isin(order_id)]['标题']
            nums = items_orders[items_orders['订单编号'].isin(order_id)]['购买数量']
            df_user = pd.concat([pd.Series(items),pd.Series(nums)],axis=1)
            df_user.groupby(by='标题').apply(sum) # 按照物品分组合计购买数量
            df_user['user']=user
            df_user.reset_index(drop=True,inplace=True)
            df = pd.concat([df,df_user]) # 组合每个
            df.reset_index(drop=True,inplace=True)
            
        itemid = []
        for item in df['标题']:
            itemid.append(list(items_attribute[items_attribute['标题'].isin([item])]['宝贝ID'])) # 将标题替换成宝贝ID
            
        itemids = [] # 整理宝贝ID 空列表以Nan替代
        for item_id in itemid:
            if item_id:
                itemids.append(item_id[0])
            else:
                itemids.append(np.nan)
        
        df['宝贝ID']=itemids
        
        # 删除标题列 删除空值
        df.drop(columns='标题',inplace=True)
        df.dropna(inplace=True)
        df.reset_index(drop=True,inplace=True) # 重置索引
        df['购买数量'] = MinMaxScaler().fit_transform(np.array(df['购买数量']).reshape(-1,1)) * 10 # 归一化 0-10
        self.data = [[df['user'][i],df['宝贝ID'][i],df['购买数量'][i]] for i in range(len(df))] #构建列表 1，1，4；1，5，1... 
        
        
        def compress(data,col):
            """
            处理data数据第col列的数据，保证此列数字是从0开始连续出现，中间不会有一个不存在此列的数字
            :param data: 二维列表数据
            :param col:处理的列
            :return:不同的个数
            """

            e_rows = dict()  # key是data数据中第col列数据，value是一个存放键出现在每一个行号的列表
            for i in range(len(data)):
                e = data[i][col]
                if e not in e_rows:
                    e_rows[e] = []
                e_rows[e].append(i)

            for rows,i in zip(e_rows.values(),range(len(e_rows))):
                for row in rows:
                    data[row][col] = i

            return len(e_rows)
        
        self.num_user = compress(self.data,0)
        self.num_item = compress(self.data,1)
        print('num_user:{},num_item:{}'.format(self.num_user,self.num_item))
        
        # 训练集和测试集
        self.train, self.test = self.__split_data()
        print('总共有{}条数据，训练集{}条，测试集{}条，用户{}个，物品{}个'.format(len(self.data),
                                                                                  len(self.train), len(self.test),
                                                                                  self.num_user, self.num_item))
        
    def __split_data(self):
        """
        将数据进行随机划分8分，1份作为测试集，7份作为训练集
        return:训练集和测试集
        """
        
        print('split_data...')
        test = []
        train = []
        for user, item, num in self.data:
            if random.randint(1, 8) == 1:
                test.append([user, item, num])  # 测试集
            else:
                train.append([user, item, num])  # 训练集

        return train, test
        
        

In [90]:
data = Data()

开始读取数据...
num_user:3320,num_item:255
split_data...
总共有20489条数据，训练集17831条，测试集2658条，用户3320个，物品255个


In [15]:
data.data

[[0, 0, 12],
 [0, 1, 20],
 [0, 2, 16],
 [0, 3, 10],
 [0, 4, 1],
 [1, 5, 1],
 [1, 5, 1],
 [2, 6, 1],
 [3, 7, 1],
 [4, 8, 8],
 [5, 9, 1],
 [5, 10, 1],
 [5, 11, 1],
 [5, 7, 1],
 [5, 12, 1],
 [5, 13, 1],
 [5, 14, 1],
 [5, 15, 1],
 [5, 16, 1],
 [6, 17, 6],
 [6, 18, 4],
 [6, 19, 2],
 [6, 20, 4],
 [6, 21, 3],
 [6, 22, 3],
 [6, 23, 2],
 [6, 24, 3],
 [7, 1, 10],
 [7, 22, 16],
 [8, 25, 60],
 [8, 25, 48],
 [8, 25, 48],
 [9, 26, 1],
 [9, 27, 1],
 [9, 27, 1],
 [9, 28, 1],
 [9, 7, 1],
 [9, 29, 1],
 [9, 30, 5],
 [9, 4, 1],
 [10, 31, 2],
 [10, 32, 1],
 [10, 33, 1],
 [10, 34, 2],
 [10, 35, 1],
 [10, 36, 1],
 [10, 37, 1],
 [10, 38, 1],
 [10, 38, 1],
 [10, 38, 1],
 [10, 38, 1],
 [11, 25, 36],
 [11, 5, 4],
 [11, 5, 4],
 [11, 5, 5],
 [11, 5, 4],
 [11, 5, 5],
 [11, 21, 5],
 [11, 1, 5],
 [12, 39, 1],
 [12, 40, 1],
 [12, 21, 1],
 [12, 41, 1],
 [13, 42, 1],
 [14, 43, 1],
 [15, 7, 3],
 [15, 7, 4],
 [16, 43, 12],
 [17, 44, 1],
 [18, 45, 3],
 [18, 46, 5],
 [18, 47, 1],
 [18, 48, 6],
 [18, 49, 5],
 [18, 49, 5],
 [

## UserCF

In [16]:
# UserCF

In [128]:
# 封装推荐指数函数
def cal_recommend_index(k,user_id,item_id,freq_matrix,user_similar_matrix):
    """函数功能：传入用户ID，产品ID，计算该产品对于用户的推荐指数
    参数：k:取几个最相近用户
          user_id：用户ID：给指定用户进行推荐
          item_id：产品ID
          freq_matrix：评分矩阵
          user_similar_matrix：相似度矩阵    
    """
    # 提取当前想要推荐用户的评分向量
    user_id_action = freq_matrix[user_id]
    
    # 所有用户对于产品3的评分向量
    item_id_action = freq_matrix[:,item_id]
    
    # 用户user_id对所有用户的相似度
    user_id_similar = user_similar_matrix[user_id]
    
    # 前k个（数组中-(k+1)：-1）最相似的用户提取出来
    similar_index = np.argsort(user_id_similar)[-(k+1):-1]
    
    # 定义两个变量额，存储用户的分数,产品的权重
    score = 0 # 分子 用户分数
    weight = 0 # 分母 产品权重

    # 对相近的k个用户进行遍历
    for similar_user_id in similar_index: #[2,1]
        #print(similar_user_id)

        # 先判断这个用户有没有对这个产品进行评分，如果没有打分，则直接计算下一个用户
        if item_id_action[similar_user_id] != 0:
            # 用户和当前用户的相似度 * 用户对这个产品的打分
            score += user_id_similar[similar_user_id] * item_id_action[similar_user_id] 
            #print('用户ID:',similar_user_id,'用户分数之和:',score)

            # 计算公式的分母，即所有用户相似度之和
            weight += user_id_similar[similar_user_id]
            #print('相似度之和:',weight)

    # 相似的k个用户是没有对这个产品进行评分的
    if score == 0:
        # print(0)
        pass
    else:
        # 最终推荐指数
        rec_index = score / weight
        return rec_index

In [114]:
def cal_recommend_index_matrix(freq_matrix,user_similar_matrix):
    """函数功能：传入参数相似度矩阵和用户评分矩阵，计算返回用户推荐矩阵
    参数：
    freq_matrix：用户评分矩阵
    user_similar_matrix：相似度矩阵
    返回：计算的推荐系统矩阵
    """

    user_count = freq_matrix.shape[0] # 获取用户数
    item_count = freq_matrix.shape[1] # 获取商品数

    # 构建一个用户数*商品数的空矩阵
    predit_matrix = np.zeros((user_count,item_count))


    for user_id in range(user_count):
        for item_id in range(item_count):

            # 首先判断原来的评分矩阵中有没有值的存在，如果有不需要计算 即买过了不推荐
            if freq_matrix[user_id,item_id] == 0:

                # 调用推荐指数函数，返回值填充到predit_matrix对应位置
                predit_matrix[user_id,item_id] = cal_recommend_index(k=5,
                                                                     user_id=user_id,
                                                                     item_id=item_id,
                                                                     freq_matrix=freq_matrix,
                                                                     user_similar_matrix=user_similar_matrix)
    return predit_matrix

In [93]:
def usercf_recommend(data):
    df = pd.DataFrame(data,columns=['userid','itemid','buy_num'])
    #df['buy_num'] = MinMaxScaler().fit_transform(np.array(df['buy_num']).reshape(-1,1)) * 10 # 归一化 0-10
    # 创建矩阵
    df_pivot = df.pivot_table(index='userid',columns='itemid',values='buy_num') # 重塑数据
    freq = df_pivot.fillna(0) # 空值填充
    # 转换成数组方便计算
    freq_matrix = freq.values
    user_similar_matrix = cosine_similarity(freq_matrix) # 计算相似度
    
    predit_matrix = cal_recommend_index_matrix(freq_matrix,user_similar_matrix)
    # 将推荐矩阵转换成dataframe
    recommend_df = pd.DataFrame(predit_matrix,index=df_pivot.index,columns=df_pivot.columns)
    # 堆叠在一起三列
    recommend_df_result = recommend_df.stack().reset_index()
    recommend_df_result.rename(columns={0:'推荐指数'},inplace=True) # 替换列名
    return recommend_df_result

In [94]:
usercf_recommendation = usercf_recommend(data.train[:100])
usercf_recommendation

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


,userid,itemid,推荐指数
0,0,1,0.000000
1,0,2,0.000000
2,0,3,0.000000
3,0,5,0.048641
4,0,21,0.057225
...,...,...,...
99,22,57,0.014306
100,22,60,0.000000
101,22,61,0.014306
102,22,68,0.000000


In [135]:
def get_recommendation(data,user,item_k):
    data = pd.DataFrame(data,columns=['userid','itemid','buy_num'])
    #newuser = pd.DataFrame(user,columns=['userid','itemid','buy_num'])
    #alluser = pd.concat([train,newuser])
    #alluser['buy_num'] = MinMaxScaler().fit_transform(np.array(alluser['buy_num']).reshape(-1,1)) * 10 # 归一化 0-10
    # 创建矩阵
    df_pivot = data.pivot_table(index='userid',columns='itemid',values='buy_num') # 重塑数据
    freq = df_pivot.fillna(0) # 空值填充
    # 转换成数组方便计算
    freq_matrix = freq.values
    user_similar_matrix = cosine_similarity(freq_matrix) # 计算相似度

    user_count = 1 # 获取新用户的推荐商品
    item_count = freq_matrix.shape[1] # 获取商品数
    user_id = user
    # 构建一个用户数*商品数的空矩阵
    predit_matrix = np.zeros((user_count,item_count))
    for item_id in range(item_count):
        # 首先判断原来的评分矩阵中有没有值的存在，如果有不需要计算 即买过了不推荐
        if freq_matrix[0,item_id] == 0:
            # 调用推荐指数函数，返回值填充到predit_matrix对应位置
            predit_matrix[0,item_id] = cal_recommend_index(k=5,
                                                         user_id=user_id,
                                                         item_id=item_id,
                                                         freq_matrix=freq_matrix,
                                                         user_similar_matrix=user_similar_matrix)
    recommend_df = pd.DataFrame(predit_matrix,columns=df_pivot.columns)
    # 堆叠在一起三列
    recommend_df_result = recommend_df.stack().reset_index()
    # 对列名进行修改
    recommend_df_result.rename(columns={0:'推荐指数'},inplace=True) # 替换列名
    # 排序获取最推荐的k个
    topk = set(recommend_df_result.sort_values(by='推荐指数',ascending=False)['itemid'][:item_k])
    return topk

In [96]:
topk = get_recommendation(data.train[:100],[[0, 0, 12]],10)
topk

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


{0, 1, 2, 3, 5, 21, 22, 25}

In [133]:
def evaluate(data):
    # 得到测试集用户与其他所有正反馈物品集合的映射
    test_user_items = dict()
    
    for user,item,rank in data.test:
        if user not in test_user_items:
            test_user_items[user] = set()

        test_user_items[user].add(item)

    # 计算准确率和召回率

    hit = 0 # 分子
    all_ru = 0 # 分母
    all_tu = 0 # 分母
    

    
    for user,item,rank in data.test:
        # 训练集数据给用户的推荐列表
        ru = get_recommendation(data.data,user,10)
        # 测试集数据给用户的推荐列表
        tu = test_user_items[user]

        hit += len(ru & tu)
        all_ru += len(ru)
        all_tu += len(tu)

    return hit / all_ru, hit / all_tu

In [134]:
def UserCF_test():
    precisions = []
    recalls = []
    times = []
    
    data = Data()
    startTime = time.time()
    
    usercf_recommendation = usercf_recommend(data.data)
    precision,recall = evaluate(data)
    
    times.append('%.3fs'%(time.time()-startTime))
    precisions.append('%.3f%%'%(precision * 100))
    recalls.append('%.3f%%'%(recall * 100))
    
    df = pd.DataFrame()
    df['algorithm'] = ['UserCF']
    df['precision'] = precision
    df['recall'] = recall
    df['time'] = times
    
    #print(df)
    return usercf_recommendation,df

In [136]:
usercf_recommendation, df = UserCF_test()

开始读取数据...
num_user:3320,num_item:255
split_data...
总共有20489条数据，训练集17916条，测试集2573条，用户3320个，物品255个


In [137]:
usercf_recommendation

,userid,itemid,推荐指数
0,0,0,0.000000
1,0,1,0.000000
2,0,2,0.000000
3,0,3,0.000000
4,4,8,0.000000
...,...,...,...
15738,3319,149,0.000000
15739,3319,152,0.014306
15740,3319,156,0.057225
15741,3319,164,0.000000


In [138]:
df

,algorithm,precision,recall,time
0,UserCF,0.08566,0.234183,603.511s


## ItemCF

In [139]:
class ItemCF:
    
    def __init__(self,data):
        """
        基于物品的协同过滤算法
        :pama data：无上下文信息的隐性反馈数据集，包含训练集和测试集
        """
        self.data = data
        
        print('开始计算物品相似度矩阵')
        self.W = self.__item_similarity()
        
        self.K = None # 推荐时，选择与物品最相似的物品个数
        self.N = None # 每个用户最多推荐的物品个数
        
        self.recommendation = None # 每个用户的推荐列表
    
    def compute_recommendation(self,K=10,N=10):
        """
        计算推荐列表
        :param K:推荐时，选择与物品最相似的物品个数
        :param N:每个用户最多推荐的物品个数
        """
        self.N = N
        self.K = K
        
        print('开始计算推荐列表(K='+str(self.K)+',N='+str(self.N)+')')
        
        self.recommendation = self.__get_recommendation()
    
    
    
    def __item_similarity(self):
        """
        计算训练集的用户相似度
        return:存放两个用户之间相似度二维列表
        """
        #  train_user_items[i] 是用户i有过正反馈的所有物品列表
        train_user_items = [[] for u in range(self.data.num_user)]
        
        for user,item in self.data.train:
            train_user_items[user].append(item)
        
        print('统计每两个物品之间的共同有过正反馈的用户数量和每个物品又被正反馈的总量')
        
        # W[i][j] 物品i和物品j共同被正反馈的数量(j>i)
        W = [[0 for j in range(self.data.num_item)] for i in range(self.data.num_item)]
        
        N = [0 for i in range(self.data.num_item)] # N[i] 代表物品i有过正反馈的数量
        
        for items in train_user_items:
            for item in items:
                # 统计N
                N[item] += 1
                
                # 统计W
                for j in items:
                    if j > item:
                        W[item][j] += 1
                        
        print('计算每两个物品之间的相似度')
        
        for i in range(self.data.num_item - 1):
            for j in range(i+1,self.data.num_item):
                if W[i][j] != 0:
                    W[i][j] = W[i][j] / math.sqrt(N[i] * N[j])
                    W[j][i] = W[i][j]
                    
        return W
        
        
    def __get_recommendation(self):
        """
        得到所有用户的推荐物品列表
        return:推荐列表
        """
        
        # 每个用户所有有过正反馈的物品集合
        train_user_items = [set() for u in range(self.data.num_user)]
        for user,item in self.data.train:
            train_user_items[user].add(item)
            
        print('得到每个用户与最相近的K个物品集合')
        k_items = []
        
        for i in range(self.data.num_item):
            Wi = dict() # Wi[j] 是物品i与物品j之间的相似性
            for j in range(self.data.num_item):
                if self.W[i][j] != 0:
                    Wi[j] = self.W[i][j]
                    
            k_items.append(
                set(items[0] for items in sorted(Wi.items(),
                                                 key=operator.itemgetter(1),
                                                 reverse=True)[:self.K]))
                    
        print('计算每个用户推荐列表')
        
        recommendation = []
        for user_item_set in train_user_items:
            recommendation.append(self.__recommend(user_item_set,k_items))
        return recommendation
            
            
    def __recommend(self,user_item_set,k_items):
        """
        对每个用户没有正反馈的物品选取最相近的K个物品计算兴趣，给用户推荐最多N个物品
        
        :param user_item_set: 训练集用户所有正反馈的物品集合
        :param k_items: k_items[i] 是与物品i最相似的K个物品集合
        
        return: 推荐给用户的物品列表
        """
        
        rank = dict()
        
        for i in user_item_set:
            for j in k_items[i]:
                if j not in user_item_set:
                    rank[j] = rank.setdefault(j,0) + self.W[i][j]
        
        # 前十个推荐物品
        return [items[0] for items in sorted(rank.items(),
                                              key=operator.itemgetter(1),
                                              reverse=True)[:self.N]]

In [148]:
data.data = [[user,item] for user, item, score in data.data] # 获取无反馈信息数据
data.train = [[user,item] for user, item, score in data.train]
data.test = [[user,item] for user, item, score in data.test]

In [151]:
item = ItemCF(data)

开始计算物品相似度矩阵
统计每两个物品之间的共同有过正反馈的用户数量和每个物品又被正反馈的总量
计算每两个物品之间的相似度


In [154]:
item.compute_recommendation()

开始计算推荐列表(K=10,N=10)
得到每个用户与最相近的K个物品集合
计算每个用户推荐列表


In [155]:
print(item.recommendation)

[[87, 18, 9, 110, 25, 5, 69, 54, 220, 103], [72, 104, 148, 152, 6, 4, 70, 76, 25, 124], [5, 84, 104, 45, 76, 64, 43, 72, 44, 134], [], [104, 149, 152, 5, 1, 91, 18, 45, 161, 141], [130, 25, 87, 5, 72, 60, 54, 22, 148, 135], [104, 5, 91, 25, 54, 9, 152, 1, 43, 130], [104, 25, 54, 69, 3, 110, 27, 18, 87, 72], [130, 9, 22, 87, 5, 104, 177, 76, 1, 7], [5, 130, 104, 22, 60, 25, 9, 87, 243, 136], [104, 40, 149, 41, 130, 122, 175, 76, 164, 25], [104, 9, 72, 152, 87, 76, 148, 130, 18, 91], [104, 122, 149, 5, 9, 31, 36, 49, 35, 25], [153, 193, 240, 191, 98, 25, 235, 211, 224, 81], [91, 76, 5, 18, 45, 104, 47, 6, 66, 9], [25, 87, 104, 9, 60, 22, 130, 5, 134, 184], [91, 76, 5, 18, 45, 104, 47, 6, 66, 9], [5, 65, 58, 55, 66, 104, 70, 6, 93, 45], [5, 104, 9, 130, 87, 25, 22, 184, 72, 152], [4, 239, 243, 91, 227, 119, 251, 220, 104, 253], [104, 194, 72, 4, 70, 44, 76, 148, 184, 152], [104, 54, 25, 27, 72, 76, 118, 5, 11, 7], [104, 9, 87, 110, 5, 130, 25, 152, 22, 1], [72, 104, 148, 152, 6, 4, 70, 76

In [156]:
class Evaluation:
    
    def __init__(self,recommend_algorithm):
        """
        对推荐算法计算各种评测指标
        
        :param :recommend_algorithm : ItemCF 
        """
        
        self.rec_alg = recommend_algorithm # 算法
        
        self.precision = None # 准确率
        self.recall = None # 召回率
    
    def evalute(self):
        """
        评估指标计算
        """
        # 准确率和召回率
        self.precision,self.recall = self.__precision_recall()
        print('准确率 = ' + str(self.precision * 100) + '% 召回率 = ' + str(self.recall * 100) + '%')
    
    
    def __precision_recall(self):
        """
        计算准确率和召回率
        
        return； 准确率和召回率
        """
        
        # 得到测试集用户与其他所有正反馈物品集合的映射
        test_user_items = dict()
        
        for user,item in self.rec_alg.data.test:
            if user not in test_user_items:
                test_user_items[user] = set()
                
            test_user_items[user].add(item)
            
        # 计算准确率和召回率
        
        hit = 0 # 分子
        all_ru = 0 # 分母
        all_tu = 0 # 分母
        
        for user, item in test_user_items.items():
            # 训练集数据给用户的推荐列表
            ru = set(self.rec_alg.recommendation[user])
            # 测试集数据给用户的推荐列表
            tu = item
            
            hit += len(ru & tu)
            all_ru += len(ru)
            all_tu += len(tu)
            
        return hit / all_ru, hit / all_tu

In [158]:
algorithms = [ItemCF]
    
precisions = []
recalls = []
times = []

data = Data()
data.data = [[user,item] for user, item, score in data.data] # 获取无反馈信息数据
data.train = [[user,item] for user, item, score in data.train]
data.test = [[user,item] for user, item, score in data.test]

for algorithm in algorithms:

    startTime = time.time()
    recommend = algorithm(data)
    recommend.compute_recommendation()
    eva = Evaluation(recommend)
    eva.evalute()

    times.append('%.3fs'%(time.time()-startTime))
    precisions.append('%.3f%%'%(eva.precision * 100))
    recalls.append('%.3f%%'%(eva.recall * 100))

df = pd.DataFrame()
df['algorithm'] = [algorithm.__name__ for algorithm in algorithms]
df['precision'] = precisions
df['recall'] = recalls
df['time'] = times

print(df)

开始读取数据...
num_user:3320,num_item:255
split_data...
总共有20489条数据，训练集17962条，测试集2527条，用户3320个，物品255个
开始计算物品相似度矩阵
统计每两个物品之间的共同有过正反馈的用户数量和每个物品又被正反馈的总量
计算每两个物品之间的相似度
开始计算推荐列表(K=10,N=10)
得到每个用户与最相近的K个物品集合
计算每个用户推荐列表
准确率 = 1.9442292796281953% 召回率 = 10.436590436590437%
  algorithm precision   recall    time
0    ItemCF    1.944%  10.437%  0.210s


In [159]:
df

,algorithm,precision,recall,time
0,ItemCF,1.944%,10.437%,0.210s
